In [2]:
# pip install --upgrade langchain

In [3]:
# !pip install bitsandbytes

In [4]:
# pip install accelerate

In [5]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

# 配置BitsAndBytes的設定，用於模型的量化以提高效率。
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 啟用4位元量化
    bnb_4bit_compute_dtype=torch.float16,  # 計算時使用的數據類型
    bnb_4bit_quant_type="nf4",  # 量化類型
    bnb_4bit_use_double_quant=True,  # 使用雙重量化
)

# 定義模型ID，用於從HuggingFace Hub加載模型。
model_id = "Open-Orca/Mistral-7B-OpenOrca"

# 加載並配置模型，這裡使用了前面定義的量化配置。
model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # 自動選擇運行設備
    quantization_config=quantization_config,
)

# 加載模型的分詞器。
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 創建一個用於文本生成的pipeline。
text_generation_pipeline = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=3200,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# 創建一個HuggingFacePipeline實例，用於後續的語言生成。
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

c:\Users\USER\Desktop\SQL\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\USER\Desktop\SQL\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipel

In [6]:
template = '''### Instruction:
Given a prompt delimited by triple backticks, perform the following actions: 
- Convert the following natural language input into a Cypher query.
- You should ignore the curly braces in the cypher query and only take question as input
- You should ignore the input keys 

### Examples:


### prompt: Retrieve all nodes connected to 'Harry Potter' with any relationship.

### output: 
MATCH (harry:Object {{name: 'Harry Potter'}})--(connectedNodes) 
RETURN connectedNodes.

### prompt: Get the names of all objects related to 'Hogwarts' with the relationship 'LOCATED_AT'.

### output: 
MATCH (object)-[:LOCATED_AT]->(hogwarts:Object {{name: 'Hogwarts'}}) 
RETURN object.name.
Ground truth information available: 

### prompt:```{question}```

### output:
'''


In [10]:
template = """Given a prompt delimited by triple backticks, perform the following actions:
- transform the prompt into a cypher query language, do not output other sentences, only cypher query language
- output the cypher query language only, do not output other sentences 
- you only need to output the answer about prompt question, and eliminate the \n in the end

###prompt: ```{question}```
###output:
"""

In [11]:
def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [15]:
result = generate_response("Who are the members of Ravenclaw?")
result

"Given a prompt delimited by triple backticks, perform the following actions:\n- transform the prompt into a cypher query language, do not output other sentences, only cypher query language\n- output the cypher query language only, do not output other sentences \n- you only need to output the answer about prompt question, and eliminate the \n in the end\n\n###prompt: ```Who are the members of Ravenclaw?```\n###output:\n```MATCH (n:Character) WHERE n.house = 'Ravenclaw' RETURN n```"

In [14]:
import gc
model = None
gc.collect() 
torch.cuda.empty_cache() # PyTorch thing

### LM studio API

In [16]:
template = """Given a prompt delimited by triple backticks, perform the following actions:
- transform the prompt into a cypher query language
- output the cypher query language
###prompt: ```{question}```
###output:
"""

In [18]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
  messages=[
    {"role": "system", "content": 
     '''Given a prompt delimited by triple backticks, perform the following actions:
    - output the cypher query language only, do not output other sentences 
    - you only need to output the answer about prompt question, and eliminate the \n in the end 
    '''},
    {"role": "user", "content": "You are given a knowledge graph. Note that the relations character is all lower and with no space in between and the node is Object with a name label. Use the provided knowledge graph to construct cypher query to answer the following question. Who are the members of Ravenclaw?"}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" ```cpp\nMATCH (h:House)-[:HAS_MEMBER]->(m:Member)\nWHERE h.name = 'Ravenclaw'\nRETURN m.name\n```", role='assistant', function_call=None, tool_calls=None)
